# Awk

[GNU Gawk User's Manual](https://www.gnu.org/software/gawk/manual/gawk.html)

[Awk on Wikipedia](https://en.wikipedia.org/wiki/AWK)

##### Tutorial Derived from the following:
http://linuxcommand.org/lc3_adv_awk.php

## History

The AWK programming language is truly one of the classic tools used in Unix. It dates back to the very earliest days of the Unix tradition. It was originally developed in the late 1970's at Bell Telephone Laboratories by Alfred Aho, Peter Weinberger, and Brian Kernighan. The name "AWK" comes from the last names of the three authors. It underwent major improvements in 1985 with the release of nawk or "new awk." It is that version that we still use today, though it is usually just called awk.

## So, What's It Good For?

Though AWK is fairly general purpose, it is really designed to create filters, that is, programs that accept standard input, transform data, and send it to standard output. In particular, AWK is very good at processing columnar data. This makes it a good choice for developing report generators, and tools that are used to re-format data. Since it has strong regular expression support, it's good for very small text extraction and reformatting problems, too. Like sed, many AWK programs are just one line long.

In recent years, AWK has fallen out of fashion, being supplanted by other, newer, interpreted languages such as Perl and python, but AWK still has some advantages:

 - It's easy to learn. The language is not overly complex and has a syntax much like the C programming language, so learning it will be useful in the future when we study other languages and tools.

 - It really excels at a solving certain types of problems.

## How It Works

 - Operates on a single `record` at a time
 
 - A `record` is a single line of text terminate by a newline character `\n`
 
 - Each `record` is separated into `fields`, which by default are separated by white space
 
 - Fields are given numeric variables: `$1`, `$2`, `$3`, `$4`, `$5`.....and so on and so on.
 
 - The field `$0` is special and representes the entire `record`.

In [ ]:
awk --version

### The first example

In [ ]:
ls -l /usr/bin | awk '{print $0}'

### Breaking down the first example

- The awk program is contained within the single quotation marks `'{print $0}'`

- Quotation marks prevent the shell from expanding the `$` symbol

- The output of `ls -l /usr/bin` is piped into our program line by line, i.e. record by record.

- `{print $0}` prints out the entire record.

In [ ]:
# print the first field

ls -l /usr/bin | awk '{print $1}'

In [ ]:
# print the ninth field

ls -l /usr/bin | awk '{print $9}'

Most lines returned by `ls -l /usr/bin` have nine fields separated by whitespace.

But some files in `/usr/bin` are symbolically links to other files, which show up in subsequent fields.

Let's find these by print all records where the number of fields exceeds 9.

In [ ]:
ls -l /usr/bin | awk 'NF > 9 {print $0}'

### Special patterns

Awk patterns can run in many forms, such as conditional expressions in the above example and regular expressions.

Two special patterns exist: `BEGIN` and `END`

- `BEGIN` carries out its action before the first record is read
- `END` carries out its action after the last record is read

### Special Pattern Example

##### Building a directory report

Let's employ `BEGIN` and `END` to build a directory report from `ls -l /usr/bin`. 

Steps:

1) Use `BEGIN` to make a header.

```
Directory Report
================
```

2) For each record that has more than nine fields, use this pattern: `{print $9, "is a symbolic link to", $NF}`

3) Create a footer at the very end of the report.
```
=============
End Of Report
```

In [ ]:
# Print a directory report

ls -l /usr/bin | awk '
    BEGIN {
        print "Directory Report"
        print "================"
    }

    NF > 9 {
        print $9, "is a symbolic link to", $NF
    }

    END {
        print "============="
        print "End Of Report"
    }

'

### Relational expressions

Relational expressions are used to test fields for values. For example, we can test for equivalence:

`$1 == "Fedora"`

or for relations such as:

`$3 >= 50`

It is also possible to perform calculations like:

`$1 * $2 < 100`

### pattern logical-operator pattern

It is possible to combine patterns together using the logical operators || and &&, meaning OR and AND, respectively.

For example, to test a record to see if the first field is greater than 100 and the last field is "Debit":

`$1 > 100 && $NF == "Debit"`

## Built-in Variables

### `FS`

##### Field separator

By default, awk automatically separates fields by whitespace. The field separator `FS` variable can be set for other field separators.

```
BEGIN { FS = ":" }
{ print $1, $5 }
```

### `RS`

##### Record separator

Records are automatically separated by the newline character `\n`, but can also be specified by the record separator variable `RS`. For example, if records are separated by empty lines, then we modify our `RS` to be:

```
BEGIN { FS = "\n"; RS = "" }
```

### `NF`

##### Number of fields

This variable updates each time a record is read. We can easily access the last field in the record by referring to `$NF`.

### `OFS`

##### Output field separator

This string is used to separate fields when printing output. The default is a single space. Setting this can be handy when reformatting data, e.g.,

In [ ]:
ls -l /usr/bin | awk 'BEGIN {OFS = ","} NF == 9 {print $1,$2,$3,$4,$5,$6,$7,$8,$9}'

### `ORS`

##### Output record separator

This is the string used to separate records when printing output. The default is a newline character. We could use this variable to easily double-space a file by setting ORS to equal two newlines:

In [ ]:
ls -l /usr/bin | awk 'BEGIN {ORS = "\n\n"} {print}'

### `FILENAME`

If AWK is reading its input from a file specified on the command line, then this variable contains the name of the file.

### `FNR `

##### File record number

When reading input from a file specified on the command line, AWK sets this variable to the number of the record read from that file.

### Example: Filtering for American airports from `airports.dat`

Let's build an awk program that filters an airport database for all USA airports and aggregates them into an organized table.

In [ ]:
# first examine the structure of the database

head airports.dat

For illustration purposes, let's report where each USA airport is located.

Example:

```
Dallas Love Field is located in Dallas
```

In [ ]:
# Print a directory report

cat airports.dat | awk '
    
    # create header
    BEGIN { FS = ","
        print "USA Airports"
        print "================"
    }

    # relational expressions
    $4 == "\"United States\"" {
        gsub("\"","",$2) # built-in string function -- here we are stripping the quotes
        gsub("\"","",$3) # built-in string function -- here we are stripping the quotes
        print $2, "is located in", $3
    }

    # create footer
    END {
        print "============="
        print "End Of Report"
    }
'

### More Complex Relational Expressions

Let's create a new report where only Canadian airports between 40 deg and 50 deg N latitude are listed.

Let's also format each record so that they appear like so:

```
Airport: ICAO Full Name
```

In [ ]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # create header
    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
    }

    # Three-conditions linked with pattern-logical operators
    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        print "Airport:", $6, $2 
    }

    # create footer
    END {
        print "==============================================="
        print "End Of Report"
    }
'

## Awk Programming
 
### User-defined functions

### Variables

##### Variables can be set in awk.

For example, we can treat a string of numbers as number by applying math to it.

```
deg = 46 + 0
min = 29 + 0
sec = 6 + 0
```

We can also force awk to treat a variable as a string by concatenating an empty string.

```
deg = 46 ""
```

### Arrays

Single-dimensional arrays are allowed in awk, and can be declared using the format:

```
array[key] = value
```

The key can be an integer or a string.

Example:

```
dms_lat["deg"] = 0 + 0
dms_lat["min"] = 0 + 0
dms_lat["sec"] = 0 + 0

dms_lon["deg"] = 0 + 0
dms_lon["min"] = 0 + 0
dms_lon["sec"] = 0 + 0
```

Arrays should be declared in the `BEGIN` pattern of your awk script.

```
BEGIN { FS = ","
    print "Canada Airports Between 40th and 50th parallels"
    print "==============================================="

    dms_lat["deg"] = 0 + 0
    dms_lat["min"] = 0 + 0
    dms_lat["sec"] = 0 + 0

    dms_lon["deg"] = 0 + 0
    dms_lon["min"] = 0 + 0
    dms_lon["sec"] = 0 + 0     
}
```

### User-defined functions

Awk allows for custom functions in a similar style to C.

Scalar-variables, strings and numbers, are passed by value, meaning they are copied into a local variable within the function.

Arrays are passed by reference, i.e. they act like global variables.

#### Example: A function that converts decimals coordinate to degrees-minutes-seconds

The scalar variables `coord_lat` and `coord_lon` are treated like local variables inside the function.

`dms_lat` is an existing array that is passed by reference and modified by the function.

```
function dec_dms(coord_lat, coord_lon) {
    dms_lat["deg"] = int(coord_lat)
    rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
    dms_lat["min"] = int(rem_deg*60)
    rem_min = rem_deg*60 - dms_lat["min"]
    dms_lat["sec"] = int(rem_min*60)

    dms_lon["deg"] = int(coord_lon)
    rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
    dms_lon["min"] = int(rem_deg*60)
    rem_min = rem_deg*60 - dms_lon["min"]
    dms_lon["sec"] = int(rem_min*60)        
}
```

### Example: Converting Airport Decimal Coordinates to Degrees-Minutes-Seconds

In [ ]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can define functions in Awk
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)
        
        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)        
    }

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format
    
        dms_lat["deg"] = 0 + 0
        dms_lat["min"] = 0 + 0
        dms_lat["sec"] = 0 + 0

        dms_lon["deg"] = 0 + 0
        dms_lon["min"] = 0 + 0
        dms_lon["sec"] = 0 + 0     
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        # print only the airport ICAO code and lat/long in DMS format
        print "Airport: ", $6, dms_lat["deg"] "\xc2\xb0", dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0", dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

### Expanding Our Example

##### Conditional statements

In [ ]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0 + 0
        dms_lat["min"] = 0 + 0
        dms_lat["sec"] = 0 + 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0 + 0
        dms_lon["min"] = 0 + 0
        dms_lon["sec"] = 0 + 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        print "Airport: ", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

### Expanding Once More

##### printf "print format"

In [ ]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0
        dms_lat["min"] = 0
        dms_lat["sec"] = 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0
        dms_lon["min"] = 0
        dms_lon["sec"] = 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        printf "%-5s %-5s %-4s %-4s %-5s %-4s %-4s \n", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

## Saving Our Script to An Awk Script File

In [ ]:
echo '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0
        dms_lat["min"] = 0
        dms_lat["sec"] = 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0
        dms_lon["min"] = 0
        dms_lon["sec"] = 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        printf "%-5s %-5s %-4s %-4s %-5s %-4s %-4s \n", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
' > awk_prog.awk

In [ ]:
cat awk_prog.awk

In [ ]:
cat airports.dat | awk -f awk_prog.awk